In [6]:
import pandas as pd

df3 = pd.read_csv('class-grades.csv')
print(df3.describe())


df3.loc[df3['Final'].isnull()==True,'Missing']="Y"
df3.loc[df3['Final'].isnull()==False,'Missing']="N"
print(df3.loc[df3['Missing']=="Y",'Missing'])


#dfassign=df3.pivot(columns = 'Missing',values=['Assignment'])['Assignment']
#print(dfassign['Y'])

#dfassign.boxplot(column=['Y','N'],grid=False)

#将 Midterm 按 Missing（Final 是否缺失）分类：
dfassign=df3.pivot(columns = 'Missing',values=['Midterm'])['Midterm']
#print(dfassign['Y'])

#目的是查看 Midterm 是否影响 Final 的缺失情况。
dfassign.boxplot(column=['Y','N'],grid=False)
#d.boxplot(column=['A', 'B', 'C', 'D'], grid=False)
#plt.show()

"""
三种缺失值（MCAR、MAR、MNAR）对应的插补方法
1. 适用于 MCAR（完全随机缺失）
MCAR（Missing Completely at Random）：缺失值与任何变量无关，数据的缺失是完全随机的。
插补方法：
均值/中位数/众数填充（Mean/Median/Mode Imputation）：简单且不会引入系统性偏差，适用于数值型或类别型数据。
删除缺失数据（Listwise or Pairwise Deletion）：如果缺失值较少，直接删除不会影响数据的代表性。

2. 适用于 MAR（随机缺失）
MAR（Missing at Random）：缺失值与其他可观测变量相关，但与自身值无关。
插补方法：
回归插补（Regression Imputation）：利用其他变量预测缺失值，适用于数值型数据。
多重插补（MICE）（Multiple Imputation by Chained Equations）：模拟多个插补数据集，并结合不确定性进行估计。

3. 适用于 MNAR（非随机缺失）
MNAR（Missing Not at Random）：缺失值与自身值相关，即使控制其他变量，仍然存在系统性缺失。
插补方法：
KNN 近邻插补（K-Nearest Neighbors Imputation）：基于最相似的样本进行插补，适用于数据模式明显的情况。
最大似然估计（MLE）（Maximum Likelihood Estimation）：基于统计模型估计缺失值，适用于有明显分布假设的数据。
使用外部数据或专家知识：由于 MNAR 可能隐藏重要信息，最好尽量获取额外数据，而不是简单插补。

"""






import pandas as pd
import numpy as np

from statsmodels.api import add_constant
import statsmodels.discrete.discrete_model as sml
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

candidates = {'Ed_standard': [780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
              'Work_experience':[5.1,4.5,None,3.3,3.6,9.3,6.7,2.8,5.4,None,7.8,None,None,10.1,6.7,None],
              'Salary': [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000],
              'happiness': [0.5,0.55,0.1,0.6,0.7,0.45,0.56,0.73,0.45,0.67,0.43,0.23,0.78,0.42,0.36,0.23]
              }
df = pd.DataFrame(candidates,columns= ['Ed_standard','Salary','Work_experience','happiness'])

df.loc[df['Work_experience'].isnull()==False,'Missing']=0
df.loc[df['Work_experience'].isnull()==True,'Missing']=1

print(df.describe())
corr = df[['Ed_standard','Salary','Work_experience','happiness']].corr()
corr.style.background_gradient(cmap='coolwarm')


#df.to_csv('candidates.csv')#保存成 CSV 文件



X=df[['Ed_standard','Salary']]
X = add_constant(X)

y=df['Missing']

logit = sml.Logit(y, X).fit()
print(logit.summary())  #LLR p-value（似然比检验的 p 值）= 0.2225，表示整体模型的统计显著性不强  R²（Pseudo R²）= 0.1512 这个值较低，说明 Ed_standard 和 Salary 对 Missing 的解释能力不强。
#print(logit.predict())

"""逻辑回归（Logistic Regression），用于分析 Work_experience 变量的缺失（Missing=1）是否与 Ed_standard（教育标准）和 Salary（薪资）相关。

结论
Work_experience 的缺失（Missing=1）与 Ed_standard 和 Salary 没有显著关系：

逻辑回归分析表明，这两个变量的 p 值 > 0.05，无法预测 Work_experience 是否缺失。
这表明 Work_experience 可能是 MCAR（完全随机缺失），而不是 MAR（随机缺失）。
如果 Work_experience 是 MCAR，可以直接删除缺失值或使用均值填充：

由于缺失值与 Ed_standard 和 Salary 没有关系，可以直接删除缺失数据或使用简单插补（如均值或中位数填充）。
如果 Work_experience 影响后续分析，可以使用 KNN 或 MICE 进行插补：

如果数据是 MAR，建议使用回归插补或 MICE（MICE - Multiple Imputation by Chained Equations），保持数据的关系结构。
如果数据是 MNAR，建议使用KNN 或机器学习方法，避免插补带来的系统性偏差。。


"""



"""简单插补 Univariate Imputation 适用于MCAR（完全随机缺失）"""

import numpy as np
from sklearn.impute import SimpleImputer

y=np.array([[780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
    [5.1,4.5,np.nan,3.3,3.6,9.3,6.7,2.8,5.4,np.nan,7.8,np.nan,np.nan,10.1,6.7,np.nan],
    [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000],
    [0.5,0.55,0.1,0.6,0.7,0.45,0.56,0.73,0.45,0.67,0.43,0.23,0.78,0.42,0.36,0.23]])
#y=np.reshape(y, (4, 16))
y=y.transpose()
#填补np.nan，使用mean  strategy='median'
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
#imp.fit(y)
y=imp.fit_transform(y)
print(y)


import statsmodels.formula.api as sm
import statsmodels.stats.stattools as st
import statsmodels.stats.api as sms
import pandas as pd
df=pd.DataFrame(y)
df.columns=['X1','X2','X3','Y']

formula_str="Y~X1 + X2 +X3"

result=sm.ols(formula=formula_str,data=df).fit()
print(result.summary())

"""观察填补 X2 缺失值后，看回归模型的系数和显著性是否受到影响。
如果 X2 影响很大，可能需要不同的填补策略，比如 median（中位数填补）或 KNN（最近邻填补）。"""




"""IterativeImputer迭代插补 属于多变量插补 Multivariate Imputation，使用贝叶斯岭回归（BayesianRidge）作为回归模型
IterativeImputer迭代插补 是 Scikit-learn 实现的 MICE（MICE - Multiple Imputation by Chained Equations）变体 """

import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd

from sklearn.impute import SimpleImputer
#方法一，形状难看
y=np.array([[780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
    [5.1,4.5,np.nan,3.3,3.6,9.3,6.7,2.8,5.4,np.nan,7.8,np.nan,np.nan,10.1,6.7,np.nan],
    [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000],
    [0.5,0.55,0.1,0.6,0.7,0.45,0.56,0.73,0.45,0.67,0.43,0.23,0.78,0.42,0.36,0.23]])
#y=np.reshape(y, (4, 16))
y=y.transpose()
#random_state=0
imp = IterativeImputer(max_iter=100, sample_posterior=True,tol=0.000001)
y=imp.fit_transform(y)
print(y)

#方法2，形状好看
y=np.array([[780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
    [5.1,4.5,np.nan,3.3,3.6,9.3,6.7,2.8,5.4,np.nan,7.8,np.nan,np.nan,10.1,6.7,np.nan],
    [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000],
    [0.5,0.55,0.1,0.6,0.7,0.45,0.56,0.73,0.45,0.67,0.43,0.23,0.78,0.42,0.36,0.23]])
y = y.T  # 转置数据，使形状匹配
# 使用 IterativeImputer 进行插补
imp = IterativeImputer(max_iter=100, sample_posterior=True, tol=1e-6, random_state=0)
y_imputed = imp.fit_transform(y)
# 转换为 pandas DataFrame 并格式化显示
df = pd.DataFrame(y_imputed, columns=["Feature_1", "Feature_2", "Feature_3", "Feature_4"])
print(df)

"""IterativeImputer（迭代插补）是一种 多变量插补 方法，它使用 其他特征来预测缺失值，不像 SimpleImputer 仅使用均值或中位数。

工作原理：使用贝叶斯岭回归（BayesianRidge）作为回归模型
选择 一个 变量作为 y（需要填补的特征）。
其他变量作为 X（用来预测 y）。
用 X 训练一个回归模型来预测 y 中的 np.nan。
对所有变量重复这个过程，迭代 max_iter 次，最终返回最优结果。

让插补过程带有一定的随机性，每次运行都会略微不同。
这样可以模拟数据的不确定性，但也意味着结果不稳定
"""



"""KNNImputer（基于 K 近邻的插补）"""

import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
y=np.array([[780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
    [5.1,4.5,np.nan,3.3,3.6,9.3,6.7,2.8,5.4,np.nan,7.8,np.nan,np.nan,10.1,6.7,np.nan],
    [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000],
    [0.5,0.55,0.1,0.6,0.7,0.45,0.56,0.73,0.45,0.67,0.43,0.23,0.78,0.42,0.36,0.23]])

#y=np.array([[780,750,690,710,680,730,690,720,740,900,950,975,995,1000,1010,1020],
#    [5.1,4.5,np.nan,3.3,3.6,9.3,6.7,2.8,5.4,np.nan,7.8,np.nan,np.nan,10.1,6.7,np.nan],
#    [78000,75000,100000,71000,68000,70000,69000,72000,74000,69000,102000,101000,79000,114000,101000,95000]])

#y=np.reshape(y, (4, 16))
y=y.transpose()
#random_state=0

imputer = KNNImputer(n_neighbors=2,weights="distance")
y_res=imputer.fit_transform(y)
# the model learns that the second feature is double the first
print(y_res)


#在 KNNImputer 里，Y 也是一个特征变量，因此：如果 Y 也有缺失值，KNN 可能会错误地用 X1, X2, X3 来预测 Y，
#去掉 Y 变量后插补
X = y[:, :-1]  # 只使用 X1, X2, X3 进行插补
imputer = KNNImputer(n_neighbors=2, weights="distance")
X_res = imputer.fit_transform(X)

# 重新组合 X 和 Y
y_res = np.hstack([X_res, y[:, -1].reshape(-1, 1)])
print(y_res)


"""KNN 插补后，重新回归分析"""
import statsmodels.formula.api as sm
import pandas as pd

df = pd.DataFrame(y_res, columns=['X1', 'X2', 'X3', 'Y'])  # 转换为 DataFrame

formula_str = "Y ~ X1 + X2 + X3"
result = sm.ols(formula=formula_str, data=df).fit()
print(result.summary())

#表明插入数据后更好了
#Log-Likelihood = 12.260	对数似然函数值提高了（之前是 11.472），说明拟合更好。
#AIC = -16.52, BIC = -13.43	AIC/BIC 变小，表明模型改进。
#Prob(JB) = 0.987 残差分布接近正态
#Durbin-Watson = 1.890	接近 2，说明残差无明显自相关问题。


from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[['X1', 'X2', 'X3']]
X['Intercept'] = 1  # 添加常数项
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)






"""Outlier"""

from sklearn.datasets import fetch_california_housing
import numpy as np
import pandas as pd

housing = fetch_california_housing()  # 载入加州房价数据集
print(housing.DESCR)  # 输出数据集描述


x = housing.data  # 特征数据
y = housing.target  # 房价

columns = housing.feature_names  # 特征名称
housing_df = pd.DataFrame(housing.data)  # 转换为 DataFrame

housing_df.columns = columns  # 设置列名
housing_df['Y'] = pd.Series(y)  # 添加目标变量（房价） 把 y 转换为 pandas Series，并添加到 DataFrame
housing_df

import seaborn as sns
import numpy as np
sns.boxplot(x=housing_df['AveOccup']) #boxplot看Outlier  这是 look for univariate outliers 的方法


import matplotlib.pyplot as plt
import numpy as np

x = np.array(housing_df.index.tolist())  # X轴：索引值      将 DataFrame 的索引转换为 Python list，然后再转为 NumPy array。
y1 = np.array(housing_df['AveOccup'])  # Y轴：每户平均居住人数  默认是 pandas.Series，需要转换为 NumPy array 以便 matplotlib 正确处理。

f = plt.figure()  # 创建画布
ax = f.add_subplot(111)  # 添加子图

plt.plot(x, y1)  # 绘制折线图
plt.axhline(y=housing_df['AveOccup'].mean())  # 添加均值线
plt.axhline(y=housing_df['AveOccup'].mean() + 3 * housing_df['AveOccup'].std(), color='r')  # 添加 +3σ 线
plt.axhline(y=housing_df['AveOccup'].mean() - 3 * housing_df['AveOccup'].std(), color='r')  # 添加 -3σ 线

plt.title('Individual Charts', fontsize=8)  # 设置标题
plt.show()  # 显示图表



""" Multivariate outlier"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.stats import f
from sklearn.datasets import fetch_california_housing

# 加载加州房价数据集
housing = fetch_california_housing()
data = housing.data  # 特征数据
target = housing.target  # 目标变量（房价中位数）

# 将数据转换为 Pandas DataFrame
housing_df = pd.DataFrame(data, columns=housing.feature_names)

# 计算数据的行数（样本数）和列数（变量数）
n = len(housing_df)  # 样本数量
k = len(housing_df.columns)  # 变量数量

# 计算 Hotelling's T² 统计量的 99% 临界值  #Hotelling's T² 统计量用于检测多变量数据中的异常点。
Hotvalue = f.ppf([0.99], n, k) * (n * k) / ((n - k) + 1)
print("Hotelling's T² 临界值:", Hotvalue) # critical value（38.88768082）  Hotvalue是一个数组，而不是一个单一数值

# 计算数据的逆协方差矩阵（用于 Mahalanobis 距离计算）
inverse_cov = np.linalg.pinv(housing_df.cov().values)

# 计算数据的均值向量
xbar = housing_df.mean().values

# 计算每个数据点的 Hotelling's T² 统计量    和inverse_cov一起体现多变量
hotelling_scores = []
for i in range(n):
    mahalanobis_dist = distance.mahalanobis(housing_df.iloc[i], xbar, inverse_cov) #Mahalanobis 距离 是一种 多变量尺度（Multivariate Distance）
    t2_value = (n * k) * mahalanobis_dist ** 2 / (k * (n - k))
    hotelling_scores.append(t2_value)

# 将 Hotelling's T² 统计量添加到 DataFrame
housing_df['hotelling'] = hotelling_scores
housing_df['critical value'] = Hotvalue[0] #如果 hotelling（T² 值） > critical value（38.88768082），则该点可能是异常值。

# 绘制 Hotelling's T² 统计量的可视化图表
plt.figure(figsize=(10, 5))
plt.plot(range(n), hotelling_scores, label='Hotelling T² Values')
plt.axhline(y=housing_df['hotelling'].mean(), color='g', linestyle='--', label='Mean T²')
plt.axhline(y=Hotvalue, color='r', linestyle='-', label='99% Critical Value')
plt.title('Multivariate Outlier Detection Using Hotelling’s T² Test')
plt.xlabel('Sample Index')
plt.ylabel('Hotelling’s T² Value')
plt.legend()
plt.show()

# 输出异常值（超出临界值的数据点）
outliers = housing_df[housing_df['hotelling'] > housing_df['critical value']]
print("异常值:")
print(outliers)



"""分箱  Binning Methods for Data Smoothing"""
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# 加载 Iris 数据集
dataset = load_iris()
data = dataset.data  # 取所有数值特征
feature_names = dataset.feature_names  # 特征名称
df = pd.DataFrame(data, columns=feature_names)

# 选择某个特征列（例如：sepal width (cm)）
col = df.iloc[:, 1]  # 选择第二列特征（花萼宽度）

# 按照分箱数量排序
sorted_col = np.sort(col.values)

# 设定分箱数量（如 30 组，每组 5 个数据点）
num_bins = 30
bin_size = len(sorted_col) // num_bins  # 每个 bin 的大小

# **1. 均值分箱**
bin_means = sorted_col.copy()
for i in range(0, len(sorted_col), bin_size):
    bin_means[i:i+bin_size] = np.mean(sorted_col[i:i+bin_size])

print("Bin Mean Smoothing:\n", bin_means)

# **2. 边界分箱**
bin_boundaries = sorted_col.copy()
for i in range(0, len(sorted_col), bin_size):
    min_val = sorted_col[i]
    max_val = sorted_col[i+bin_size-1] if i+bin_size-1 < len(sorted_col) else sorted_col[-1]
    for j in range(bin_size):
        if abs(sorted_col[i+j] - min_val) < abs(sorted_col[i+j] - max_val):
            bin_boundaries[i+j] = min_val
        else:
            bin_boundaries[i+j] = max_val

print("Bin Boundary Smoothing:\n", bin_boundaries)

# **3. 中位数分箱**
bin_medians = sorted_col.copy()
for i in range(0, len(sorted_col), bin_size):
    median_val = np.median(sorted_col[i:i+bin_size])
    bin_medians[i:i+bin_size] = median_val

print("Bin Median Smoothing:\n", bin_medians)




       Ed_standard         Salary  Work_experience  happiness    Missing
count    16.000000      16.000000        11.000000   16.00000  16.000000
mean    833.750000   83625.000000         5.936364    0.48500   0.312500
std     136.607711   15564.382416         2.420443    0.19225   0.478714
min     680.000000   68000.000000         2.800000    0.10000   0.000000
25%     717.500000   70750.000000         4.050000    0.40500   0.000000
50%     765.000000   76500.000000         5.400000    0.47500   0.000000
75%     980.000000  100250.000000         7.250000    0.61750   1.000000
max    1020.000000  114000.000000        10.100000    0.78000   1.000000
Optimization terminated successfully.
         Current function value: 0.527167
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                Missing   No. Observations:                   16
Model:                          Logit   Df Residuals:                       13
Meth